In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### 데이터 읽어오기

In [ ]:
def extract(df):
    X = df.drop(columns=['MEDV'])
    Y = df['MEDV']
    return X, Y

# 데이터 읽기
data = pd.read_csv('data/train.csv').set_index('ID')
test = pd.read_csv('data/test.csv').set_index('ID')
input_x, input_y = extract(data)
submit_x = test

### 검증

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# 검증
model = RandomForestRegressor(random_state=42)
rmse_scores = -cross_val_score(model, input_x, input_y, scoring='neg_root_mean_squared_error', cv=5)

# 평균 RMSE 출력
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")

### 훈련 및 예측 파일 생성

In [ ]:
# 훈련
model = RandomForestRegressor(random_state=42)
model.fit(input_x, input_y)

# 결과 파일 생성
submit_y = pd.DataFrame(model.predict(submit_x), index=submit_x.index, columns=['MEDV'])
submit_y.to_csv("submission.csv", index=True)